#

<div align=center>
<img src="https://uol.unifor.br/acesso/app/autenticacao/assets/img/logos/icon-unifor.svg" width=45 height=45>
<img src="https://vortex.unifor.br/assets/logos/v1.png" width=45 height=45>
<font size=5 color=k>
<br><br>
<font size=5 color=k><strong>Projeto:</strong> Doctor Bone

<strong>Etapa:</strong> Treinamento e teste

<strong>Autoria:</strong> Heitor Teixeira

</div>

## 
<font size=5 color='blue'> 0 - Bibliotecas e configuração de GPU

In [ ]:
import tensorflow as tf
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import GPUtil
from tensorflow.keras.optimizers import Adam, SGD, RMSprop # type: ignore
from tensorflow.keras.losses import MeanAbsoluteError # type: ignore
from auxiliar.dataset import criar_dataset
from auxiliar.avaliar import avaliar_modelo
from auxiliar.callbacks import callbacks
from auxiliar.arquiteturas import custom_model

tf_gpu = tf.config.experimental.list_physical_devices('GPU')
if tf_gpu:
    try:
        nome_gpu = GPUtil.getGPUs()
        print("GPU configurada: ", nome_gpu[0].name)
        tf.config.experimental.set_memory_growth(tf_gpu[0], True)
    except RuntimeError as e:
        print(e)


## 
<font size=5 color='blue'> 1 - Hiperparametros

In [ ]:
'''
ao lado o batchsize que comporta na minha placa de vídeo, adequa com o que couber.
'''

redes = [
        'InceptionV3', # 100
        'InceptionResNetV2', # 50
        'EfficientNetV2L', # 5
        'EfficientNetV2S', # 16
        'ResNet152V2', # 50
        'EfficientNetV2M', # 10
        'EfficientNetV2B3', # 50
        'ResNet50V2', # 150
   ]

In [ ]:
'''
parametros iniciais para o treinamento, incluindo o identificador de rodada,
a rede a ser usada, o batch size e o numero de epocas. definir tambem a funcao de perda, 
o otimizador e outros parametros relevantes.

lembrar de ajustar a lr quando mudar de otimizador e rede. quanto maior e mais camadas tiver uma rede, menor deve ser a lr
tem a questão de explodir os gradientes e estabilidade. quanto mais profunda for a rede menor a lr.
na maioria das vezes da pra usar o adam com lr = 0.001. mas se for prauma efficientnet M L ou XL é bom dar uma diminuida.
'''

seed=2220276
id_rodada = 0
rede = redes[3]
batch_size = 16

epocas = 100
loss = MeanAbsoluteError() 
nome_loss = 'mae' # uma string pra salvar no csv
lr = 0.0001

'''
aqui tao os parametros da arquitetura do modelo. alguns desses parametros fazem nda,
é so uma string pra salvar no log e a gente saber que aquele resultado tinha determinada conf.
daria pra colocar uma condicional na funcao de avaliar e gravar o csv? dava, mas dava mais trabalho
'''

otimizador = Adam(learning_rate = lr) # Adam, SGD, RMSprop
nome_otimizador = 'adam' # uma string pra salvar no csv

attention = 'cbam' # 'se' ou 'cbam'. as 2 camadas de attention que adicionei caso queria treinar com
camada_pooling = 'global_avg' # 'avg', 'max', 'global_max', 'global_avg', flatten(sem pooling)
imagenet = 'imagenet-21k' # imagenet, imagenet-21k, imagenet-21k-ft1k, os modelos efficient net podem ser treinados na 21k

'''
proejetei pra aceitar ate 5 camadas densas, é so ajustar que da pra aumentar. so acho que nao precisa
'''

denses = [1024, 256, 256] 
dropouts = [0, 0, 0] 

data_aug = False

check_best = True
check_10 = True
early_stop = True
log = True
reduce_lr = True


## 
<font size=5 color='blue'> 2 - Importando dados e criando datasets

In [ ]:
'''
carregar os dados de treino, validacao e teste a partir de um arquivo csv e dividir em conjuntos
de treino, validacao e teste com proporcoes definidas.

aqui eu juntei todos os dados em um so csv e em uma so pasta. no RSNA existem 3 dados, 3 arquivos zips.
1 de treino e mais 2 de validacao. juntei todos e adicionei mais outras 1388 imagens jpg que achei.
so depois de tudo junto que fiz a divisao do que é treino, validacao ou teste
'''

dir_dados = '../treino-validacao-teste.csv'
dir_imagens = '../imagens/imagens'

dir_modelos_salvos = f'./redes/{rede}/modelos_salvos/{rede}-{id_rodada}_epoca_{{epoch:02d}}.hdf5'
dir_csv_log_treino = f'./redes/{rede}/log_treino/{rede}-{id_rodada}_treinamento_log.csv'

teste_size = 0.15
val_size = 0.10

dados_treino = pd.read_csv(dir_dados)

dados_treino, dados_teste = train_test_split(
    dados_treino, 
    random_state=seed,
    test_size = teste_size,
)

dados_treino, dados_validacao = train_test_split(
    dados_treino,
    random_state=seed,
    test_size = val_size,
)

print(len(dados_treino))
print(len(dados_validacao))
print(len(dados_teste))

In [ ]:
'''
dataset é uma colecaoo estruturada de dados onde cada linha representa uma observação e cada coluna representa um atributo. 
é como se fosse um dado tabular, mas ele é otiimzados pro treinamento
diferenca de um dataframe:

a funcao ja foi comentada nos arquivos auxiliares
'''

dataset_treino = criar_dataset(
    dataframe = dados_treino,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,
    shuffle = True,
    data_aug = data_aug
)

dataset_validacao = criar_dataset(
    dataframe = dados_validacao,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,
)

dataset_teste = criar_dataset(
    dataframe = dados_teste,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,
)

## 
<font size=5 color='blue'> 3 - Modelando a rede

In [ ]:
# ja foi comentado nos arquivos auxiliares

model = custom_model(
    rede= rede,
    loss = loss,
    weights = imagenet,
    otimizador = otimizador, 
    attention = attention, 
    denses = denses,
    dropouts = dropouts,
    pooling = camada_pooling
)

'''
aqui é so pra gente ter nocao do quao grande é o modelo que a gente vai treinar.
vai me dizer quantas camadas o modelo vai ter e sumario bem direitinho dele
'''
num_layers = len(model.layers)
print(f'camadas no modelo: {num_layers}')

model.summary()

In [ ]:
# ja foi comentado nos arquivos auxiliares

callbacks_treino = callbacks(
    dir_modelos_salvos = dir_modelos_salvos, 
    dir_csv_log = dir_csv_log_treino,
    check_best = check_best, 
    check_15 = check_10, 
    early_stop = early_stop, 
    log = log, 
    reduce_lr = reduce_lr,
    )

## 
<font size=5 color='blue'> 4 - Treinar modelo

In [ ]:
# ja foi comentado nos arquivos auxiliares

steps_per_epoch = len(dados_treino) // batch_size
validation_steps=len(dados_validacao) // batch_size

start_time = time.time()

model.fit(
    dataset_treino, 
    validation_data = dataset_validacao, 
    steps_per_epoch = steps_per_epoch, 
    validation_steps = validation_steps, 
    epochs = epocas,
    callbacks=callbacks_treino
    )

end_time = time.time()

tempo_treino = end_time - start_time
print(tempo_treino)

## 
<font size=5 color='blue'> 5 - Avaliar modelo

In [ ]:
# ja foi comentado nos arquivos auxiliares

avaliar_modelo(
    model=model,
    rede=rede,
    id_rodada=id_rodada,  
    dataset_teste=dataset_teste,
    dados_teste=dados_teste,
    batch_size=batch_size,
    imagenet=imagenet,
    data_aug=data_aug,
    nome_otimizador=nome_otimizador,
    nome_loss=nome_loss,
    lr=lr,
    attention=attention,
    camada_pooling=camada_pooling,
    denses=denses,
    dropouts=dropouts,
    tempo_treino=tempo_treino
)
